In [7]:
import pandas as pd
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

In [8]:
bert_df = pd.read_csv('/Users/esauhutcherson/ResumeMatcher/esau/data_results/Bert_Tokenizer_BM25_matches.csv')

In [9]:
bert_df

,Resume_str,Job_posting,BM25_score
0,"DIRECTOR, GLOBAL BUSINESS DEVELOPMENT...","Director of Sales, Natural Resources Company O...",753.715336
1,"DIRECTOR, GLOBAL BUSINESS DEVELOPMENT...","Director, North America Business Product Owner...",751.546740
2,"DIRECTOR, GLOBAL BUSINESS DEVELOPMENT...","Senior Manager, Wireless Infrastructure Respon...",750.559579
3,"DIRECTOR, GLOBAL BUSINESS DEVELOPMENT...",Commercial Sales Manager (TX) SolarEdge (NASDA...,750.128456
4,"DIRECTOR, GLOBAL BUSINESS DEVELOPMENT...","Senior Product Marketing Manager, ID NOW and C...",746.909609
...,...,...,...
195,ENGLISH LANGUAGE ARTS TEACHER S...,Mastery Teaching Residency About Mastery:\n\nF...,566.415662
196,ENGLISH LANGUAGE ARTS TEACHER S...,Middle School Science Teacher Company Descript...,565.267507
197,ENGLISH LANGUAGE ARTS TEACHER S...,"K-12 Teacher - Elementary, Middle, High School...",564.991259
198,ENGLISH LANGUAGE ARTS TEACHER S...,Middle School Math Teacher Company Description...,559.086483


In [10]:
if 'eval_score' not in bert_df.columns:
    bert_df['eval_score'] = None

In [ ]:
resume_index = 0
job_index = 0
total_jobs = len(bert_df)
jobs_per_resume = 10
total_resumes = total_jobs // jobs_per_resume

progress_bar = widgets.IntProgress(value=0, min=0, max=total_jobs, description='Progress:', bar_style='info')

def get_df_idx():
    return resume_index * jobs_per_resume + job_index

def render():
    clear_output()

    if resume_index >= total_resumes:
        progress_bar.value = total_jobs
        display(progress_bar)
        display(HTML("<h3 style='color: green;'>✅ Evaluation complete. CSV saved as <code>manual_evaluation_done.csv</code>.</h3>"))
        return

    df_idx = get_df_idx()

    resume_text = bert_df.loc[df_idx, 'Resume_str']
    job_text = bert_df.loc[df_idx, 'Job_posting']
    bm25_score = bert_df.loc[df_idx, 'BM25_score']

    progress_bar.value = df_idx
    display(progress_bar)

    display(HTML(f"""
    <div style="white-space: pre-wrap; font-size: 18px; border: 2px solid #ccc; padding: 10px; margin-top: 10px; margin-bottom: 10px;">
        <b>Resume {resume_index + 1}:</b><br><br>{resume_text}
    </div>
    """))

    display(HTML(f"""
    <div style="white-space: pre-wrap; font-size: 16px; border: 1px solid #aaa; padding: 8px; margin-bottom: 10px;">
        <b>Job Match {job_index + 1} of 10 (BM25 Score: {bm25_score:.2f}):</b><br><br>{job_text}
    </div>
    """))

    relevant_button.disabled = False
    not_relevant_button.disabled = False
    display(widgets.HBox([relevant_button, not_relevant_button]))

def save_eval_and_advance(score):
    global job_index, resume_index

    df_idx = get_df_idx()
    bert_df.at[df_idx, 'eval_score'] = score

    job_index += 1
    if job_index >= jobs_per_resume:
        job_index = 0
        resume_index += 1

    if resume_index >= total_resumes:
        bert_df.to_csv("bert_bm25_manual_evaluation.csv", index=False)

    render()

def on_relevant_clicked(b):
    relevant_button.disabled = True
    not_relevant_button.disabled = True
    save_eval_and_advance(1)

def on_not_relevant_clicked(b):
    relevant_button.disabled = True
    not_relevant_button.disabled = True
    save_eval_and_advance(0)

relevant_button = widgets.Button(description="👍 Relevant", button_style='success')
not_relevant_button = widgets.Button(description="👎 Not Relevant", button_style='danger')

relevant_button.on_click(on_relevant_clicked)
not_relevant_button.on_click(on_not_relevant_clicked)

render()


IntProgress(value=200, bar_style='info', description='Progress:', max=200)